In [1]:
import numpy as np  # linear algebra
import pandas as pd  # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn import datasets, linear_model
from datetime import datetime
import gc
%matplotlib inline
from IPython.display import display, HTML
from pprint import pprint

## Usage!!!
- Always change maxVal if iterative feature is added..5, for np.nan's add 4, then 3.. maxVal calculates according to this..

In [2]:
class ConfigElements:
    def __init__(self, lag, nameAndGroups, targetVariable="", deleteColumns = False):
        self.lag = lag
        self.nameAndGroups = nameAndGroups
        #If there is target variable, then 5 4 3 2 1, fill the np.nans..Else hold them all in Dataframe..
        self.targetVariable = targetVariable
        self.deleteColumns = deleteColumns

In [3]:
configLag0Target1 = ConfigElements(0,[ ("A",["Semana","Ruta_SAK"],["count","count"]),("B",["Ruta_SAK"],["mean"]),
    ("C",["Semana","Ruta_SAK"],["count","mean"])], "lag0tar1", False)
print  configLag0Target1.__dict__

{'nameAndGroups': [('A', ['Semana', 'Ruta_SAK'], ['count', 'count']), ('B', ['Ruta_SAK'], ['mean']), ('C', ['Semana', 'Ruta_SAK'], ['count', 'mean'])], 'lag': 0, 'targetVariable': 'lag0tar1', 'deleteColumns': False}


In [4]:
configLag2Target1 = ConfigElements(2,[ ("SA21",["Semana","Ruta_SAK"],["count"]),("L21",["Semana","Ruta_SAK"],["mean"])
                                     ],"lag1tar1")
print  configLag1Target1.__dict__

NameError: name 'configLag1Target1' is not defined

In [ ]:
configLag1Target1 = ConfigElements(1,[ ("SA2",["Semana","Ruta_SAK"],["count"]),("L",["Semana","Ruta_SAK"],["mean"])
                                     ],"lag1tar1")
print  configLag1Target1.__dict__

In [ ]:
configLag1Target1F = ConfigElements(1,[ ("SAP1CC",["Semana","Ruta_SAK","Producto_ID"],
                                    ["count"]),("SAP1MM",["Semana","Ruta_SAK","Producto_ID"],["mean"])],"Lag1Target1F",True
                                     )
print  configLag0Target1.__dict__

##  Test for filling missing values that are already on a column, no need to create a target variable..

In [5]:
#configWithoutLag2 = ConfigElements(0,[ ("A",["Agencia_ID"]), ("Ca",["Canal_ID"]), ("R",["Ruta_SAK"]), 
#    ("Cl",["Cliente_ID"]),("P",["Producto_ID"]),("ClP",["Cliente_ID","Producto_ID"])   ], "lag2par1")
#print  configWithLag.__dict__

In [6]:
#take 1 CSV, then split it to 3..
class FeatureEngineering:

    def __init__(self, ValidationStart, ValidationEnd, trainHdfPath, trainHdfFile, testHdfPath1, testHdfPath2, testHdfFile, 
                 testTypes, trainTypes, trainCsvPath, testCsvPath, maxLag=0):
        self.ValidationStart = ValidationStart
        self.ValidationEnd = ValidationEnd
        self.maxLag = maxLag
        self.trainHdfPath = trainHdfPath
        self.trainHdfFile = trainHdfFile
        self.testHdfPath1 = testHdfPath1
        self.testHdfPath2 = testHdfPath2
        self.testHdfFile = testHdfFile
        self.testTypes = testTypes
        self.trainTypes = trainTypes
        self.trainCsvPath = trainCsvPath
        self.testCsvPath = testCsvPath
        
    @staticmethod
    def __printDataFrameBasics__(data):
        display(data.head(2))
        #print data.dtypes
        gc.collect()
        print(data.info(memory_usage=True))
        
    @staticmethod    
    def changeIndexTypeToLowerMemory(data):
        ##########
        #This is very critical, i accept max number is 2^32. Also, if don't do that, memory gets so much higher..
        ##########
        #data.reset_index(inplace=True)
        #data.drop("index",axis=1, inplace=True)
        #data.index = data.index.astype('uint32')
        gc.collect()
        
    def ReadHdf(self, trainOrTestOrBoth):
        '''Reads and holds Df in object memory'''            
        if trainOrTestOrBoth == 'train' or trainOrTestOrBoth=='both':
            self.train = pd.read_hdf(self.trainHdfPath,self.trainHdfFile)
            FeatureEngineering.changeIndexTypeToLowerMemory(self.train)
            FeatureEngineering.__printDataFrameBasics__(self.train)
            
        if trainOrTestOrBoth == 'test' or trainOrTestOrBoth=='both':
            self.test1 = pd.read_hdf(self.testHdfPath1,self.testHdfFile)
            self.test2 = pd.read_hdf(self.testHdfPath2,self.testHdfFile)
            FeatureEngineering.changeIndexTypeToLowerMemory(self.test1)
            FeatureEngineering.changeIndexTypeToLowerMemory(self.test2)
            FeatureEngineering.__printDataFrameBasics__(self.test1)
            FeatureEngineering.__printDataFrameBasics__(self.test2)
        
    def ReadCsv(self, trainOrTestOrBoth):
        '''Reads and holds Df in memory'''
        if trainOrTestOrBoth == 'train' or trainOrTestOrBoth == 'both':
            self.train = pd.read_csv(self.trainCsvPath, usecols=self.trainTypes.keys(), dtype=self.trainTypes)
            self.train.reset_index(inplace=True)
            self.train.drop("index",axis=1, inplace=True)
            FeatureEngineering.changeIndexTypeToLowerMemory(self.train)
            FeatureEngineering.__printDataFrameBasics__(self.train)
        if trainOrTestOrBoth == 'test' or trainOrTestOrBoth=='both':
            tempTest = pd.read_csv(self.testCsvPath, usecols=self.testTypes.keys(), dtype=self.testTypes)
            tempTest.reset_index(inplace=True)
            tempTest.drop("index",axis=1, inplace=True)
            self.test1 = tempTest.loc[tempTest.Semana == self.ValidationStart]
            self.test2 = tempTest.loc[tempTest.Semana == self.ValidationEnd]
            del tempTest
            FeatureEngineering.changeIndexTypeToLowerMemory(self.test1)
            FeatureEngineering.changeIndexTypeToLowerMemory(self.test2)
            FeatureEngineering.__printDataFrameBasics__(self.test1)
            FeatureEngineering.__printDataFrameBasics__(self.test2)
            
    @staticmethod
    def ConvertCsvToHdf(csvPath, HdfPath, HdfName, ColumnTypeDict ):
        tempDf = pd.read_csv(csvPath, usecols=ColumnTypeDict.keys(), dtype=ColumnTypeDict,index=False)
        tempDf.to_hdf(HdfPath, HdfName, format='t')
        del tempDf
        gc.collect()
        print "ConvertCsvToHdf is done.."

    def Preprocess(self, trainOrTestOrBoth, columnFunctionTypeList):
        '''columnFunctionTypeList = [ ['C1',Func1,Type], ['C2',Func2,Type],..    ]'''
        for column, func, localType in columnFunctionTypeList:
            if trainOrTestOrBoth == 'train' or trainOrTestOrBoth =='both':
                self.train.loc[:,column] =  self.train[column].apply(func).astype(localType)
            if trainOrTestOrBoth == 'test' or trainOrTestOrBoth == 'both':
                self.test1.loc[:,column] =  self.test1[column].apply(func).astype(localType)
                self.test2.loc[:,column] =  self.test2[column].apply(func).astype(localType)
        gc.collect()
        
    def SaveDataFrameToHdf(self,trainOrTestOrBoth):
        if trainOrTestOrBoth == 'train' or trainOrTestOrBoth=='both':
            self.train.to_hdf(self.trainHdfPath, self.trainHdfFile, format='t')
        if trainOrTestOrBoth == 'test' or trainOrTestOrBoth=='both':
            self.test1.to_hdf(self.testHdfPath1, self.testHdfFile, format='t', index="False")
            self.test2.to_hdf(self.testHdfPath2, self.testHdfFile, format='t', index="False")
        
    def AddDemandaGeneralMean(self): 
        self.train.loc[:,"DemandaGeneralMean"] = self.train["Demanda_uni_equil"].loc[
            self.train.loc[:,'Semana'] < 10].mean().astype("float32")
        #self.train.loc[:,"DemandaGeneralMean"] = DemandaMeanWithoutLag['Demanda_uni_equil'].mean()
        #self.train.loc[:,"DemandaGeneralMean"] = self.train.loc[:,"DemandaGeneralMean"].astype('float32')
        #display(self.train)
        #del DemandaMeanWithoutLag
        gc.collect()
        
    '''ConfigElements(0,[ ("A",["Semana","Agencia_ID"],["count","count"]),'''
    def AddConfigurableFeaturesToTrain(self, config):
        if config.lag > self.maxLag:
            self.maxLag = config.lag
        
        tempData = self.train.loc[self.train.loc[:,'Semana'] <= self.ValidationEnd - config.lag]
        #display(tempData)
        if(config.lag != 0):
            tempData.loc[:,'Semana'] = tempData['Semana'].apply(lambda x:x + config.lag)
        #display(tempData)
        
        #Means iterative.. eliminate as long as np.nan exists..If there is already one, don't create, use the existing
        if config.targetVariable != "" and  config.targetVariable not in self.train.columns:
            self.train.loc[:,config.targetVariable] = np.nan
            self.test1.loc[:,config.targetVariable] = np.nan
            
            if config.lag != 1:
                self.test2.loc[:,config.targetVariable] = np.nan
        
        for name,groups,aggregate in config.nameAndGroups:
            if name not in self.train.columns:
                print "{} is not in columns..".format(name)            
                
                groupedDataframe = tempData[groups+['Demanda_uni_equil']].groupby(groups).agg(aggregate[0])
                #groupedDataframe.columns = groupedDataframe.columns.droplevel(0)
                groupedDataframe.columns = [name]
                
                #This is means of the counts of the semana-columns tuples!..!!!
                #If no lag and mean, mean of the columns without semana!!..
                #If there is lag and count, count of the columns x weeks before
                #If there is lag and mean, mean of the columns x weeks before
                #if(config.lag == 0 and aggregate == "count"):
                if(len(aggregate)>1):
                    groupedDataframe.reset_index(inplace=True)
                    groupedDataframe.drop("Semana",axis=1, inplace=True)
                    groups = groups[1:]
                    groupedDataframe = groupedDataframe.groupby(groups).agg(aggregate[1])
                    groupedDataframe.columns = [name]
                    gc.collect()
                
                display(groupedDataframe)
                self.train = self.train.merge( groupedDataframe, left_on=groups,
                    right_index=True, how='left', sort=False,copy=False)
                self.test1 = self.test1.merge( groupedDataframe, left_on=groups,
                    right_index=True, how='left', sort=False,copy=False)
                if config.lag != 1:
                    self.test2 = self.test2.merge( groupedDataframe, left_on=groups,
                        right_index=True, how='left', sort=False,copy=False)
                
                del groupedDataframe
                gc.collect()
            else:
                print "{} is in columns..".format(name)
            
            display(self.train)
            display(self.test1)
            display(self.test2)
            
            #Means iterative..!!!!!
            if config.targetVariable != "":
                self.train.loc[pd.isnull(self.train[config.targetVariable]), 
                    config.targetVariable] = self.train.loc[pd.isnull(self.train[config.targetVariable]), name]
                self.test1.loc[pd.isnull(self.test1[config.targetVariable]), 
                    config.targetVariable] = self.test1.loc[pd.isnull(self.test1[config.targetVariable]), name]
                if config.lag != 1:
                    self.test2.loc[pd.isnull(self.test2[config.targetVariable]), 
                        config.targetVariable] = self.test2.loc[pd.isnull(self.test2[config.targetVariable]), name]
                    
                count = self.test1.loc[:,config.targetVariable].isnull().sum()
                print "Count of missing numbers after {} in validation part 1 in column {} is {}".format(name, 
                    config.targetVariable,str(count))
                if config.lag != 1:
                    count = self.test2.loc[:,config.targetVariable].isnull().sum()
                    print "Count of missing numbers after {} in validation part 2 in column {} is {}".format(name, 
                        config.targetVariable,str(count))
                
                
                #display(self.train)
                #If column is already in Dataframe and we want to fill target variable, this deletes columns!!!
                if(config.deleteColumns):
                    self.train.drop(name, axis=1, inplace=True)
                    self.test1.drop(name, axis=1, inplace=True)
                    if config.lag != 1:
                        self.test2.drop(name, axis=1, inplace=True)
                gc.collect()
                #Only in tesst
                #if count == 0:
                 #   break
        del tempData
        display(self.train)   
        display(self.test1)   
        display(self.test2)
        gc.collect()
        return 
    
    def DeleteLaggedWeeksFromTrain(self,trainOrTestOrBoth):
        self.train = self.train.loc[self.train.loc[:,'Semana']>= 3 + self.maxLag]
        gc.collect()
        display(self.train.head(2))
        
    def ReadFirstNRowsOfACsv(self, nrows, trainOrTestOrBoth) :
        if trainOrTestOrBoth == 'train' or trainOrTestOrBoth=='both':
            self.train = pd.read_csv(self.trainCsvPath, usecols=self.trainTypes.keys(), dtype=self.trainTypes, nrows = nrows)
            FeatureEngineering.changeIndexTypeToLowerMemory(self.train)
            FeatureEngineering.__printDataFrameBasics__(self.train)
        if trainOrTestOrBoth == 'test' or trainOrTestOrBoth=='both':
            tempTest = pd.read_csv(self.testCsvPath, usecols=self.testTypes.keys(), dtype=self.testTypes, nrows = nrows*2)
            self.test1 = tempTest.loc[tempTest.Semana == self.ValidationStart]
            self.test2 = tempTest.loc[tempTest.Semana == self.ValidationEnd]
            del tempTest
            FeatureEngineering.changeIndexTypeToLowerMemory(self.test1)
            FeatureEngineering.changeIndexTypeToLowerMemory(self.test2)
            FeatureEngineering.__printDataFrameBasics__(self.test1)
            FeatureEngineering.__printDataFrameBasics__(self.test2)
    
    #Use when concatanating train and validation before predict test for example..
    def AppendTestToTrain(self,deleteTest = True):
        self.train = self.train.append(self.test1,ignore_index=True)
        gc.collect()
        if(deleteTest):
            del self.test1
            gc.collect()
        try:
            self.train = self.train.append(self.test2,ignore_index=True)
            gc.collect()
            if(deleteTest):
                del self.test2
                gc.collect()
        except:
            pass
    #Split train data to train and test1 and test2 (validation)
    def SplitTrainToTestUsingValidationStart(self):
        boolCondition = self.train.Semana == self.ValidationStart
        self.test1 = self.train.loc[boolCondition]
        self.train.drop((self.train.loc[boolCondition].index), axis=0,inplace=True)
        gc.collect()
        
        boolCondition = self.train.Semana == self.ValidationEnd
        self.test2 = self.train.loc[boolCondition]
        self.train.drop((self.train.loc[boolCondition].index), axis=0,inplace=True)
        
        del boolCondition
        gc.collect()

In [7]:
parameterDict =       {"ValidationStart":10, 
 "ValidationEnd":11,
   "maxLag":2,
    "trainHdfPath":'../../input/train.h5',
    "trainHdfFile":"train",
    "testHdfPath1":"../../input/test1.h5",
    "testHdfPath2":"../../input/test2.h5",
    "testHdfFile":"test", 
    "trainTypes" : {'Semana':np.uint8, 'Agencia_ID':np.uint16, 'Canal_ID':np.uint8,'Ruta_SAK':np.uint16, 
        'Cliente_ID':np.uint32, 'Producto_ID':np.uint16,'Venta_uni_hoy':np.uint16, 'Venta_hoy':np.float32,
                    'Dev_uni_proxima': np.uint32, 'Dev_proxima':np.float32,'Demanda_uni_equil':np.uint32}, 
    "testTypes" : {'id':np.uint32,'Semana':np.uint8, 'Agencia_ID':np.uint16, 'Canal_ID':np.uint8,'Ruta_SAK':np.uint16,
        'Cliente_ID':np.uint32, 'Producto_ID':np.uint16},
    "trainCsvPath":'../../input/train.csv'   ,
    "testCsvPath":'../../input/test.csv'}

FE = FeatureEngineering(**parameterDict)
print FE.__dict__

{'trainCsvPath': '../../input/train.csv', 'maxLag': 2, 'testTypes': {'Cliente_ID': <type 'numpy.uint32'>, 'Ruta_SAK': <type 'numpy.uint16'>, 'Canal_ID': <type 'numpy.uint8'>, 'Producto_ID': <type 'numpy.uint16'>, 'Agencia_ID': <type 'numpy.uint16'>, 'Semana': <type 'numpy.uint8'>, 'id': <type 'numpy.uint32'>}, 'testHdfFile': 'test', 'trainTypes': {'Dev_proxima': <type 'numpy.float32'>, 'Venta_uni_hoy': <type 'numpy.uint16'>, 'Cliente_ID': <type 'numpy.uint32'>, 'Demanda_uni_equil': <type 'numpy.uint32'>, 'Ruta_SAK': <type 'numpy.uint16'>, 'Canal_ID': <type 'numpy.uint8'>, 'Venta_hoy': <type 'numpy.float32'>, 'Producto_ID': <type 'numpy.uint16'>, 'Agencia_ID': <type 'numpy.uint16'>, 'Dev_uni_proxima': <type 'numpy.uint32'>, 'Semana': <type 'numpy.uint8'>}, 'testHdfPath1': '../../input/test1.h5', 'ValidationEnd': 11, 'testHdfPath2': '../../input/test2.h5', 'testCsvPath': '../../input/test.csv', 'ValidationStart': 10, 'trainHdfFile': 'train', 'trainHdfPath': '../../input/train.h5'}


In [8]:
FE.ReadHdf('train')

,Semana,Agencia_ID,Canal_ID,Ruta_SAK,Cliente_ID,Producto_ID,Venta_uni_hoy,Venta_hoy,Dev_uni_proxima,Dev_proxima,Demanda_uni_equil
0,3,1110,7,3301,15766,1212,3,25.139999,0,0.0,1.386294
1,3,1110,7,3301,15766,1216,4,33.520000,0,0.0,1.609438


<class 'pandas.core.frame.DataFrame'>
Int64Index: 74180464 entries, 0 to 74180463
Data columns (total 11 columns):
Semana               uint8
Agencia_ID           uint16
Canal_ID             uint8
Ruta_SAK             uint16
Cliente_ID           uint32
Producto_ID          uint16
Venta_uni_hoy        uint16
Venta_hoy            float32
Dev_uni_proxima      uint32
Dev_proxima          float32
Demanda_uni_equil    float32
dtypes: float32(3), uint16(4), uint32(2), uint8(2)
memory usage: 2.6 GB
None


In [14]:
FE.train.index.values.size

74180464

In [9]:
FE.train.reset_index(inplace=True)

In [10]:
FE.train.drop("index",axis=1, inplace=True)

In [13]:
FE.train.index = FE.train.astype('uint32')

In [14]:
FE.train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 74180464 entries, 0 to 74180463
Data columns (total 11 columns):
Semana               uint8
Agencia_ID           uint16
Canal_ID             uint8
Ruta_SAK             uint16
Cliente_ID           uint32
Producto_ID          uint16
Venta_uni_hoy        uint16
Venta_hoy            float32
Dev_uni_proxima      uint32
Dev_proxima          float32
Demanda_uni_equil    float32
dtypes: float32(3), uint16(4), uint32(2), uint8(2)
memory usage: 2.6 GB


In [16]:
del FE.train

In [11]:
gc.collect()

66

In [32]:
FE.ReadFirstNRowsOfACsv(1000000,'train')

,Semana,Agencia_ID,Canal_ID,Ruta_SAK,Cliente_ID,Producto_ID,Venta_uni_hoy,Venta_hoy,Dev_uni_proxima,Dev_proxima,Demanda_uni_equil
0,3,1110,7,3301,15766,1212,3,25.139999,0,0.0,3
1,3,1110,7,3301,15766,1216,4,33.520000,0,0.0,4


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000000 entries, 0 to 999999
Data columns (total 11 columns):
Semana               1000000 non-null uint8
Agencia_ID           1000000 non-null uint16
Canal_ID             1000000 non-null uint8
Ruta_SAK             1000000 non-null uint16
Cliente_ID           1000000 non-null uint32
Producto_ID          1000000 non-null uint16
Venta_uni_hoy        1000000 non-null uint16
Venta_hoy            1000000 non-null float32
Dev_uni_proxima      1000000 non-null uint32
Dev_proxima          1000000 non-null float32
Demanda_uni_equil    1000000 non-null uint32
dtypes: float32(2), uint16(4), uint32(3), uint8(2)
memory usage: 28.6 MB
None


## Append test, memory is same??

In [60]:
FE.ReadFirstNRowsOfACsv(5,'train')
#FE.train.info()
#FE.test1.info()
FE.test1 = FE.train.copy()
FE.test1["Semana"] = np.array([10,11,10,11,10]).astype("uint8")
display(FE.train)
display(FE.test1)
FE.AppendTestToTrain()
display(FE.train)
FE.train.info()
FE.test1.info()
display(FE.train)
display(FE.test1)

,Semana,Agencia_ID,Canal_ID,Ruta_SAK,Cliente_ID,Producto_ID,Venta_uni_hoy,Venta_hoy,Dev_uni_proxima,Dev_proxima,Demanda_uni_equil
0,3,1110,7,3301,15766,1212,3,25.139999,0,0.0,3
1,3,1110,7,3301,15766,1216,4,33.520000,0,0.0,4


<class 'pandas.core.frame.DataFrame'>
Int64Index: 5 entries, 0 to 4
Data columns (total 11 columns):
Semana               5 non-null uint8
Agencia_ID           5 non-null uint16
Canal_ID             5 non-null uint8
Ruta_SAK             5 non-null uint16
Cliente_ID           5 non-null uint32
Producto_ID          5 non-null uint16
Venta_uni_hoy        5 non-null uint16
Venta_hoy            5 non-null float32
Dev_uni_proxima      5 non-null uint32
Dev_proxima          5 non-null float32
Demanda_uni_equil    5 non-null uint32
dtypes: float32(2), uint16(4), uint32(3), uint8(2)
memory usage: 190.0 bytes
None


,Semana,Agencia_ID,Canal_ID,Ruta_SAK,Cliente_ID,Producto_ID,Venta_uni_hoy,Venta_hoy,Dev_uni_proxima,Dev_proxima,Demanda_uni_equil
0,3,1110,7,3301,15766,1212,3,25.139999,0,0.0,3
1,3,1110,7,3301,15766,1216,4,33.520000,0,0.0,4
2,3,1110,7,3301,15766,1238,4,39.320000,0,0.0,4
3,3,1110,7,3301,15766,1240,4,33.520000,0,0.0,4
4,3,1110,7,3301,15766,1242,3,22.920000,0,0.0,3


,Semana,Agencia_ID,Canal_ID,Ruta_SAK,Cliente_ID,Producto_ID,Venta_uni_hoy,Venta_hoy,Dev_uni_proxima,Dev_proxima,Demanda_uni_equil
0,10,1110,7,3301,15766,1212,3,25.139999,0,0.0,3
1,11,1110,7,3301,15766,1216,4,33.520000,0,0.0,4
2,10,1110,7,3301,15766,1238,4,39.320000,0,0.0,4
3,11,1110,7,3301,15766,1240,4,33.520000,0,0.0,4
4,10,1110,7,3301,15766,1242,3,22.920000,0,0.0,3


,Semana,Agencia_ID,Canal_ID,Ruta_SAK,Cliente_ID,Producto_ID,Venta_uni_hoy,Venta_hoy,Dev_uni_proxima,Dev_proxima,Demanda_uni_equil
0,3,1110,7,3301,15766,1212,3,25.139999,0,0.0,3
1,3,1110,7,3301,15766,1216,4,33.520000,0,0.0,4
2,3,1110,7,3301,15766,1238,4,39.320000,0,0.0,4
3,3,1110,7,3301,15766,1240,4,33.520000,0,0.0,4
4,3,1110,7,3301,15766,1242,3,22.920000,0,0.0,3
5,10,1110,7,3301,15766,1212,3,25.139999,0,0.0,3
6,11,1110,7,3301,15766,1216,4,33.520000,0,0.0,4
7,10,1110,7,3301,15766,1238,4,39.320000,0,0.0,4
8,11,1110,7,3301,15766,1240,4,33.520000,0,0.0,4
9,10,1110,7,3301,15766,1242,3,22.920000,0,0.0,3


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10 entries, 0 to 9
Data columns (total 11 columns):
Semana               10 non-null uint8
Agencia_ID           10 non-null uint16
Canal_ID             10 non-null uint8
Ruta_SAK             10 non-null uint16
Cliente_ID           10 non-null uint32
Producto_ID          10 non-null uint16
Venta_uni_hoy        10 non-null uint16
Venta_hoy            10 non-null float64
Dev_uni_proxima      10 non-null uint32
Dev_proxima          10 non-null float64
Demanda_uni_equil    10 non-null uint32
dtypes: float64(2), uint16(4), uint32(3), uint8(2)
memory usage: 452.0 bytes


AttributeError: FeatureEngineering instance has no attribute 'test1'

## Split Train to train and validation..Control memory usage..

In [61]:
FE.SplitTrainToTestUsingValidationStart()
FE.train.info()
FE.test1.info()
FE.test2.info()
display(FE.train)
display(FE.test1)
display(FE.test2)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5 entries, 0 to 4
Data columns (total 11 columns):
Semana               5 non-null uint8
Agencia_ID           5 non-null uint16
Canal_ID             5 non-null uint8
Ruta_SAK             5 non-null uint16
Cliente_ID           5 non-null uint32
Producto_ID          5 non-null uint16
Venta_uni_hoy        5 non-null uint16
Venta_hoy            5 non-null float64
Dev_uni_proxima      5 non-null uint32
Dev_proxima          5 non-null float64
Demanda_uni_equil    5 non-null uint32
dtypes: float64(2), uint16(4), uint32(3), uint8(2)
memory usage: 230.0 bytes
<class 'pandas.core.frame.DataFrame'>
Int64Index: 3 entries, 5 to 9
Data columns (total 11 columns):
Semana               3 non-null uint8
Agencia_ID           3 non-null uint16
Canal_ID             3 non-null uint8
Ruta_SAK             3 non-null uint16
Cliente_ID           3 non-null uint32
Producto_ID          3 non-null uint16
Venta_uni_hoy        3 non-null uint16
Venta_hoy            

,Semana,Agencia_ID,Canal_ID,Ruta_SAK,Cliente_ID,Producto_ID,Venta_uni_hoy,Venta_hoy,Dev_uni_proxima,Dev_proxima,Demanda_uni_equil
0,3,1110,7,3301,15766,1212,3,25.139999,0,0.0,3
1,3,1110,7,3301,15766,1216,4,33.520000,0,0.0,4
2,3,1110,7,3301,15766,1238,4,39.320000,0,0.0,4
3,3,1110,7,3301,15766,1240,4,33.520000,0,0.0,4
4,3,1110,7,3301,15766,1242,3,22.920000,0,0.0,3


,Semana,Agencia_ID,Canal_ID,Ruta_SAK,Cliente_ID,Producto_ID,Venta_uni_hoy,Venta_hoy,Dev_uni_proxima,Dev_proxima,Demanda_uni_equil
5,10,1110,7,3301,15766,1212,3,25.139999,0,0.0,3
7,10,1110,7,3301,15766,1238,4,39.320000,0,0.0,4
9,10,1110,7,3301,15766,1242,3,22.920000,0,0.0,3


,Semana,Agencia_ID,Canal_ID,Ruta_SAK,Cliente_ID,Producto_ID,Venta_uni_hoy,Venta_hoy,Dev_uni_proxima,Dev_proxima,Demanda_uni_equil
6,11,1110,7,3301,15766,1216,4,33.52,0,0.0,4
8,11,1110,7,3301,15766,1240,4,33.52,0,0.0,4


## Preprocess, Log("Demanda..")

In [64]:
#FE.Preprocess('both', [["Demanda_uni_equil",np.log1p,'float32']])
display(FE.train)
display(FE.test1)
display(FE.test2)

,Semana,Agencia_ID,Canal_ID,Ruta_SAK,Cliente_ID,Producto_ID,Venta_uni_hoy,Venta_hoy,Dev_uni_proxima,Dev_proxima,Demanda_uni_equil
0,3,1110,7,3301,15766,1212,3,25.139999,0,0.0,1.386294
1,3,1110,7,3301,15766,1216,4,33.520000,0,0.0,1.609438
2,3,1110,7,3301,15766,1238,4,39.320000,0,0.0,1.609438
3,3,1110,7,3301,15766,1240,4,33.520000,0,0.0,1.609438
4,3,1110,7,3301,15766,1242,3,22.920000,0,0.0,1.386294


,Semana,Agencia_ID,Canal_ID,Ruta_SAK,Cliente_ID,Producto_ID,Venta_uni_hoy,Venta_hoy,Dev_uni_proxima,Dev_proxima,Demanda_uni_equil
5,10,1110,7,3301,15766,1212,3,25.139999,0,0.0,1.386294
7,10,1110,7,3301,15766,1238,4,39.320000,0,0.0,1.609438
9,10,1110,7,3301,15766,1242,3,22.920000,0,0.0,1.386294


,Semana,Agencia_ID,Canal_ID,Ruta_SAK,Cliente_ID,Producto_ID,Venta_uni_hoy,Venta_hoy,Dev_uni_proxima,Dev_proxima,Demanda_uni_equil
6,11,1110,7,3301,15766,1216,4,33.52,0,0.0,1.609438
8,11,1110,7,3301,15766,1240,4,33.52,0,0.0,1.609438


## Save Dataframe to HDF..

In [65]:
FE.SaveDataFrameToHdf('both')

## Load from HDF, Test with big data!!!!

In [33]:
FE.ReadHdf('train')

,Semana,Agencia_ID,Canal_ID,Ruta_SAK,Cliente_ID,Producto_ID,Venta_uni_hoy,Venta_hoy,Dev_uni_proxima,Dev_proxima,Demanda_uni_equil
0,3,1110,7,3301,15766,1212,3,25.139999,0,0.0,1.386294
1,3,1110,7,3301,15766,1216,4,33.520000,0,0.0,1.609438


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 74180464 entries, 0 to 74180463
Data columns (total 11 columns):
Semana               uint8
Agencia_ID           uint16
Canal_ID             uint8
Ruta_SAK             uint16
Cliente_ID           uint32
Producto_ID          uint16
Venta_uni_hoy        uint16
Venta_hoy            float32
Dev_uni_proxima      uint32
Dev_proxima          float32
Demanda_uni_equil    float32
dtypes: float32(3), uint16(4), uint32(2), uint8(2)
memory usage: 2.1 GB
None


In [67]:
FE.train.Demanda_uni_equil = FE.train.Demanda_uni_equil.astype("uint32")

In [68]:
FE.train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5 entries, 0 to 4
Data columns (total 11 columns):
Semana               5 non-null uint8
Agencia_ID           5 non-null uint16
Canal_ID             5 non-null uint8
Ruta_SAK             5 non-null uint16
Cliente_ID           5 non-null uint32
Producto_ID          5 non-null uint16
Venta_uni_hoy        5 non-null uint16
Venta_hoy            5 non-null float64
Dev_uni_proxima      5 non-null uint32
Dev_proxima          5 non-null float64
Demanda_uni_equil    5 non-null uint32
dtypes: float64(2), uint16(4), uint32(3), uint8(2)
memory usage: 230.0 bytes


## PROD Load train and test CSV, preprocess and save as HDF..

In [8]:
FE.ReadCsv('both')

,Semana,Agencia_ID,Canal_ID,Ruta_SAK,Cliente_ID,Producto_ID,Venta_uni_hoy,Venta_hoy,Dev_uni_proxima,Dev_proxima,Demanda_uni_equil
0,3,1110,7,3301,15766,1212,3,25.139999,0,0.0,3
1,3,1110,7,3301,15766,1216,4,33.520000,0,0.0,4


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 74180464 entries, 0 to 74180463
Data columns (total 11 columns):
Semana               uint8
Agencia_ID           uint16
Canal_ID             uint8
Ruta_SAK             uint16
Cliente_ID           uint32
Producto_ID          uint16
Venta_uni_hoy        uint16
Venta_hoy            float32
Dev_uni_proxima      uint32
Dev_proxima          float32
Demanda_uni_equil    uint32
dtypes: float32(2), uint16(4), uint32(3), uint8(2)
memory usage: 2.1 GB
None


,id,Semana,Agencia_ID,Canal_ID,Ruta_SAK,Cliente_ID,Producto_ID
2,2,10,2045,1,2831,4549769,32940
7,7,10,1612,1,2837,4414012,35305


<class 'pandas.core.frame.DataFrame'>
Int64Index: 3538385 entries, 2 to 6999247
Data columns (total 7 columns):
id             uint32
Semana         uint8
Agencia_ID     uint16
Canal_ID       uint8
Ruta_SAK       uint16
Cliente_ID     uint32
Producto_ID    uint16
dtypes: uint16(3), uint32(2), uint8(2)
memory usage: 81.0 MB
None


,id,Semana,Agencia_ID,Canal_ID,Ruta_SAK,Cliente_ID,Producto_ID
0,0,11,4037,1,2209,4639078,35305
1,1,11,2237,1,1226,4705135,1238


<class 'pandas.core.frame.DataFrame'>
Int64Index: 3460866 entries, 0 to 6999250
Data columns (total 7 columns):
id             uint32
Semana         uint8
Agencia_ID     uint16
Canal_ID       uint8
Ruta_SAK       uint16
Cliente_ID     uint32
Producto_ID    uint16
dtypes: uint16(3), uint32(2), uint8(2)
memory usage: 79.2 MB
None


In [ ]:
FE.train.info()

In [9]:
FE.Preprocess('train', [["Demanda_uni_equil",np.log1p,'float32']])

In [10]:
FE.SaveDataFrameToHdf('both')

## Load train and test HDF!!

In [70]:
FE.ReadHdf('test')

,id,Semana,Agencia_ID,Canal_ID,Ruta_SAK,Cliente_ID,Producto_ID
2,2,10,2045,1,2831,4549769,32940
7,7,10,1612,1,2837,4414012,35305


<class 'pandas.core.frame.DataFrame'>
Int64Index: 3538385 entries, 2 to 6999247
Data columns (total 7 columns):
id             uint32
Semana         uint8
Agencia_ID     uint16
Canal_ID       uint8
Ruta_SAK       uint16
Cliente_ID     uint32
Producto_ID    uint16
dtypes: uint16(3), uint32(2), uint8(2)
memory usage: 81.0 MB
None


,id,Semana,Agencia_ID,Canal_ID,Ruta_SAK,Cliente_ID,Producto_ID
0,0,11,4037,1,2209,4639078,35305
1,1,11,2237,1,1226,4705135,1238


<class 'pandas.core.frame.DataFrame'>
Int64Index: 3460866 entries, 0 to 6999250
Data columns (total 7 columns):
id             uint32
Semana         uint8
Agencia_ID     uint16
Canal_ID       uint8
Ruta_SAK       uint16
Cliente_ID     uint32
Producto_ID    uint16
dtypes: uint16(3), uint32(2), uint8(2)
memory usage: 79.2 MB
None


In [73]:
FE.test1.reset_index(inplace=True)
FE.test1.drop("index",inplace=True,axis=1)

In [74]:
FE.test1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3538385 entries, 0 to 3538384
Data columns (total 7 columns):
id             uint32
Semana         uint8
Agencia_ID     uint16
Canal_ID       uint8
Ruta_SAK       uint16
Cliente_ID     uint32
Producto_ID    uint16
dtypes: uint16(3), uint32(2), uint8(2)
memory usage: 54.0 MB


In [33]:
FE.ReadFirstNRowsOfACsv(20,'train')

In [77]:
#FE.train.to_csv('../../input/train_20.csv', index= False)

## Test Setup

In [116]:
parameterDict = {"ValidationStart":8, 
    "ValidationEnd":9,
    "maxLag":2,    
    "trainHdfPath":'../../input/train_100.h5',
    "trainHdfFile":"train",
    "testHdfPath1":"../../input/train_100_1.h5",
    "testHdfPath2":"../../input/train_100_1.h5",
    "testHdfFile":"test", 
    "trainTypes" : {'Semana':np.uint8, 'Agencia_ID':np.uint16, 'Canal_ID':np.uint8,'Ruta_SAK':np.uint16, 
        'Cliente_ID':np.uint32, 'Producto_ID':np.uint16,'Venta_uni_hoy':np.uint16, 'Venta_hoy':np.float32,
                    'Dev_proxima':np.float32,'Demanda_uni_equil':np.uint32}, 
    "testTypes" : {'id':np.uint32,'Semana':np.uint8, 'Agencia_ID':np.uint16, 'Canal_ID':np.uint8,'Ruta_SAK':np.uint16,
        'Cliente_ID':np.uint32, 'Producto_ID':np.uint16},
    "trainCsvPath":'../../input/train_100.csv'   ,
    "testCsvPath":'../../input/test_100.csv'}

FE = FeatureEngineering(**parameterDict)
print FE.__dict__

{'trainCsvPath': '../../input/train_100.csv', 'maxLag': 2, 'testTypes': {'Cliente_ID': <type 'numpy.uint32'>, 'Ruta_SAK': <type 'numpy.uint16'>, 'Canal_ID': <type 'numpy.uint8'>, 'Producto_ID': <type 'numpy.uint16'>, 'Agencia_ID': <type 'numpy.uint16'>, 'Semana': <type 'numpy.uint8'>, 'id': <type 'numpy.uint32'>}, 'testHdfFile': 'test', 'trainTypes': {'Cliente_ID': <type 'numpy.uint32'>, 'Demanda_uni_equil': <type 'numpy.uint32'>, 'Producto_ID': <type 'numpy.uint16'>, 'Agencia_ID': <type 'numpy.uint16'>, 'Dev_proxima': <type 'numpy.float32'>, 'Semana': <type 'numpy.uint8'>, 'Ruta_SAK': <type 'numpy.uint16'>, 'Canal_ID': <type 'numpy.uint8'>, 'Venta_uni_hoy': <type 'numpy.uint16'>, 'Venta_hoy': <type 'numpy.float32'>}, 'testHdfPath1': '../../input/train_100_1.h5', 'ValidationEnd': 9, 'testHdfPath2': '../../input/train_100_1.h5', 'testCsvPath': '../../input/test_100.csv', 'ValidationStart': 8, 'trainHdfFile': 'train', 'trainHdfPath': '../../input/train_100.h5'}


In [117]:
FE.ReadCsv('train')
FE.Preprocess('train', [["Demanda_uni_equil",np.log1p,'float32']])
FE.SaveDataFrameToHdf('train')

,Semana,Agencia_ID,Canal_ID,Ruta_SAK,Cliente_ID,Producto_ID,Venta_uni_hoy,Venta_hoy,Dev_proxima,Demanda_uni_equil
0,3,1110,7,3301,15766,1212,3,25.139999,0.0,3
1,3,1110,7,3302,15766,1216,4,33.520000,0.0,4


<class 'pandas.core.frame.DataFrame'>
Int64Index: 28 entries, 0 to 27
Data columns (total 10 columns):
Semana               28 non-null uint8
Agencia_ID           28 non-null uint16
Canal_ID             28 non-null uint8
Ruta_SAK             28 non-null uint16
Cliente_ID           28 non-null uint32
Producto_ID          28 non-null uint16
Venta_uni_hoy        28 non-null uint16
Venta_hoy            28 non-null float32
Dev_proxima          28 non-null float32
Demanda_uni_equil    28 non-null uint32
dtypes: float32(2), uint16(4), uint32(2), uint8(2)
memory usage: 952.0 bytes
None


In [118]:
FE.ReadHdf('train')
FE.SplitTrainToTestUsingValidationStart()

,Semana,Agencia_ID,Canal_ID,Ruta_SAK,Cliente_ID,Producto_ID,Venta_uni_hoy,Venta_hoy,Dev_proxima,Demanda_uni_equil
0,3,1110,7,3301,15766,1212,3,25.139999,0.0,1.386294
1,3,1110,7,3302,15766,1216,4,33.520000,0.0,1.609438


<class 'pandas.core.frame.DataFrame'>
Int64Index: 28 entries, 0 to 27
Data columns (total 10 columns):
Semana               28 non-null uint8
Agencia_ID           28 non-null uint16
Canal_ID             28 non-null uint8
Ruta_SAK             28 non-null uint16
Cliente_ID           28 non-null uint32
Producto_ID          28 non-null uint16
Venta_uni_hoy        28 non-null uint16
Venta_hoy            28 non-null float32
Dev_proxima          28 non-null float32
Demanda_uni_equil    28 non-null float32
dtypes: float32(3), uint16(4), uint32(1), uint8(2)
memory usage: 952.0 bytes
None


In [119]:
display(FE.test1.head(2))
display(FE.test2.head(2))
display(FE.train.head(2))

,Semana,Agencia_ID,Canal_ID,Ruta_SAK,Cliente_ID,Producto_ID,Venta_uni_hoy,Venta_hoy,Dev_proxima,Demanda_uni_equil
21,8,1110,7,3305,15766,35651,12,90.0,0.0,2.564949
22,8,1110,7,3301,15766,35651,12,90.0,0.0,2.564949


,Semana,Agencia_ID,Canal_ID,Ruta_SAK,Cliente_ID,Producto_ID,Venta_uni_hoy,Venta_hoy,Dev_proxima,Demanda_uni_equil
24,9,1110,7,3302,15766,35651,12,90.0,0.0,2.564949
25,9,1110,7,3303,15766,35651,12,90.0,0.0,2.564949


,Semana,Agencia_ID,Canal_ID,Ruta_SAK,Cliente_ID,Producto_ID,Venta_uni_hoy,Venta_hoy,Dev_proxima,Demanda_uni_equil
0,3,1110,7,3301,15766,1212,3,25.139999,0.0,1.386294
1,3,1110,7,3302,15766,1216,4,33.520000,0.0,1.609438


## Test1

In [91]:
FE.AddConfigurableFeaturesToTrain(configLag0Target1)

A is not in columns..


,A
Ruta_SAK,
3301,4
3302,5
3303,3
3305,1


,Semana,Agencia_ID,Canal_ID,Ruta_SAK,Cliente_ID,Producto_ID,Venta_uni_hoy,Venta_hoy,Dev_proxima,Demanda_uni_equil,lag0tar1,A
0,3,1110,7,3301,15766,1212,3,25.139999,0.0,1.386294,NaN,4
1,3,1110,7,3302,15766,1216,4,33.520000,0.0,1.609438,NaN,5
2,3,1110,7,3303,15766,1238,4,39.320000,0.0,1.609438,NaN,3
3,3,1110,7,3301,15766,1240,4,33.520000,0.0,1.609438,NaN,4
4,3,1110,7,3302,15766,30548,3,22.920000,0.0,1.386294,NaN,5
5,4,1110,7,3302,15766,1250,5,38.200001,0.0,1.791759,NaN,5
6,4,1110,7,3302,15766,1216,3,20.280001,0.0,1.386294,NaN,5
7,4,1110,7,3301,15766,1216,6,56.099998,0.0,1.945910,NaN,4
8,4,1110,7,3301,15766,4085,4,24.600000,0.0,1.609438,NaN,4
9,5,1110,7,3302,15766,1216,6,31.680000,0.0,1.945910,NaN,5


,Semana,Agencia_ID,Canal_ID,Ruta_SAK,Cliente_ID,Producto_ID,Venta_uni_hoy,Venta_hoy,Dev_proxima,Demanda_uni_equil,lag0tar1,A
21,8,1110,7,3305,15766,35651,12,90.0,0.0,2.564949,NaN,1
22,8,1110,7,3301,15766,35651,12,90.0,0.0,2.564949,NaN,4
23,8,1110,7,3302,15766,35651,12,90.0,0.0,2.564949,NaN,5


,Semana,Agencia_ID,Canal_ID,Ruta_SAK,Cliente_ID,Producto_ID,Venta_uni_hoy,Venta_hoy,Dev_proxima,Demanda_uni_equil,lag0tar1,A
24,9,1110,7,3302,15766,35651,12,90.0,0.0,2.564949,NaN,5
25,9,1110,7,3303,15766,35651,12,90.0,0.0,2.564949,NaN,3
26,9,1110,7,3303,15766,35651,12,90.0,0.0,2.564949,NaN,3
27,9,1110,7,3305,15766,35651,12,90.0,0.0,2.564949,NaN,1


...

,Semana,Agencia_ID,Canal_ID,Ruta_SAK,Cliente_ID,Producto_ID,Venta_uni_hoy,Venta_hoy,Dev_proxima,Demanda_uni_equil,lag0tar1,A,B,C
24,9,1110,7,3302,15766,35651,12,90.0,0.0,2.564949,5.0,5,1.766936,2.0
25,9,1110,7,3303,15766,35651,12,90.0,0.0,2.564949,3.0,3,2.190761,1.0
26,9,1110,7,3303,15766,35651,12,90.0,0.0,2.564949,3.0,3,2.190761,1.0
27,9,1110,7,3305,15766,35651,12,90.0,0.0,2.564949,1.0,1,2.564949,1.0


Count of missing numbers after C in validation part 1 in column lag0tar1 is 0
Count of missing numbers after C in validation part 2 in column lag0tar1 is 0


,Semana,Agencia_ID,Canal_ID,Ruta_SAK,Cliente_ID,Producto_ID,Venta_uni_hoy,Venta_hoy,Dev_proxima,Demanda_uni_equil,lag0tar1,A,B,C
0,3,1110,7,3301,15766,1212,3,25.139999,0.0,1.386294,4.0,4,1.744863,1.75
1,3,1110,7,3302,15766,1216,4,33.520000,0.0,1.609438,5.0,5,1.766936,2.00
2,3,1110,7,3303,15766,1238,4,39.320000,0.0,1.609438,3.0,3,2.190761,1.00
3,3,1110,7,3301,15766,1240,4,33.520000,0.0,1.609438,4.0,4,1.744863,1.75
4,3,1110,7,3302,15766,30548,3,22.920000,0.0,1.386294,5.0,5,1.766936,2.00
5,4,1110,7,3302,15766,1250,5,38.200001,0.0,1.791759,5.0,5,1.766936,2.00
6,4,1110,7,3302,15766,1216,3,20.280001,0.0,1.386294,5.0,5,1.766936,2.00
7,4,1110,7,3301,15766,1216,6,56.099998,0.0,1.945910,4.0,4,1.744863,1.75
8,4,1110,7,3301,15766,4085,4,24.600000,0.0,1.609438,4.0,4,1.744863,1.75
9,5,1110,7,3302,15766,1216,6,31.680000,0.0,1.945910,5.0,5,1.766936,2.00


,Semana,Agencia_ID,Canal_ID,Ruta_SAK,Cliente_ID,Producto_ID,Venta_uni_hoy,Venta_hoy,Dev_proxima,Demanda_uni_equil,lag0tar1,A,B,C
21,8,1110,7,3305,15766,35651,12,90.0,0.0,2.564949,1.0,1,2.564949,1.00
22,8,1110,7,3301,15766,35651,12,90.0,0.0,2.564949,4.0,4,1.744863,1.75
23,8,1110,7,3302,15766,35651,12,90.0,0.0,2.564949,5.0,5,1.766936,2.00


,Semana,Agencia_ID,Canal_ID,Ruta_SAK,Cliente_ID,Producto_ID,Venta_uni_hoy,Venta_hoy,Dev_proxima,Demanda_uni_equil,lag0tar1,A,B,C
24,9,1110,7,3302,15766,35651,12,90.0,0.0,2.564949,5.0,5,1.766936,2.0
25,9,1110,7,3303,15766,35651,12,90.0,0.0,2.564949,3.0,3,2.190761,1.0
26,9,1110,7,3303,15766,35651,12,90.0,0.0,2.564949,3.0,3,2.190761,1.0
27,9,1110,7,3305,15766,35651,12,90.0,0.0,2.564949,1.0,1,2.564949,1.0


## Test 2

In [94]:
FE.AddConfigurableFeaturesToTrain(configLag2Target1 )

SA21 is not in columns..


SA21
Semana Ruta_SAK      
5      3301         2
       3302         2
       3303         1
6      3301         2
       3302         2
7      3302         3
       3303         1
8      3301         1
       3302         2
       3303         1
9      3301         2
       3302         1
       3305         1

,Semana,Agencia_ID,Canal_ID,Ruta_SAK,Cliente_ID,Producto_ID,Venta_uni_hoy,Venta_hoy,Dev_proxima,Demanda_uni_equil,lag0tar1,A,B,C,lag1tar1,SA21
0,3,1110,7,3301,15766,1212,3,25.139999,0.0,1.386294,4.0,4,1.744863,1.75,NaN,NaN
1,3,1110,7,3302,15766,1216,4,33.520000,0.0,1.609438,5.0,5,1.766936,2.00,NaN,NaN
2,3,1110,7,3303,15766,1238,4,39.320000,0.0,1.609438,3.0,3,2.190761,1.00,NaN,NaN
3,3,1110,7,3301,15766,1240,4,33.520000,0.0,1.609438,4.0,4,1.744863,1.75,NaN,NaN
4,3,1110,7,3302,15766,30548,3,22.920000,0.0,1.386294,5.0,5,1.766936,2.00,NaN,NaN
5,4,1110,7,3302,15766,1250,5,38.200001,0.0,1.791759,5.0,5,1.766936,2.00,NaN,NaN
6,4,1110,7,3302,15766,1216,3,20.280001,0.0,1.386294,5.0,5,1.766936,2.00,NaN,NaN
7,4,1110,7,3301,15766,1216,6,56.099998,0.0,1.945910,4.0,4,1.744863,1.75,NaN,NaN
8,4,1110,7,3301,15766,4085,4,24.600000,0.0,1.609438,4.0,4,1.744863,1.75,NaN,NaN
9,5,1110,7,3302,15766,1216,6,31.680000,0.0,1.945910,5.0,5,1.766936,2.00,NaN,2.0


,Semana,Agencia_ID,Canal_ID,Ruta_SAK,Cliente_ID,Producto_ID,Venta_uni_hoy,Venta_hoy,Dev_proxima,Demanda_uni_equil,lag0tar1,A,B,C,lag1tar1,SA21
21,8,1110,7,3305,15766,35651,12,90.0,0.0,2.564949,1.0,1,2.564949,1.00,NaN,NaN
22,8,1110,7,3301,15766,35651,12,90.0,0.0,2.564949,4.0,4,1.744863,1.75,NaN,1.0
23,8,1110,7,3302,15766,35651,12,90.0,0.0,2.564949,5.0,5,1.766936,2.00,NaN,2.0


,Semana,Agencia_ID,Canal_ID,Ruta_SAK,Cliente_ID,Producto_ID,Venta_uni_hoy,Venta_hoy,Dev_proxima,Demanda_uni_equil,lag0tar1,A,B,C,lag1tar1,SA21
24,9,1110,7,3302,15766,35651,12,90.0,0.0,2.564949,5.0,5,1.766936,2.0,NaN,1.0
25,9,1110,7,3303,15766,35651,12,90.0,0.0,2.564949,3.0,3,2.190761,1.0,NaN,NaN
26,9,1110,7,3303,15766,35651,12,90.0,0.0,2.564949,3.0,3,2.190761,1.0,NaN,NaN
27,9,1110,7,3305,15766,35651,12,90.0,0.0,2.564949,1.0,1,2.564949,1.0,NaN,1.0


...

,Semana,Agencia_ID,Canal_ID,Ruta_SAK,Cliente_ID,Producto_ID,Venta_uni_hoy,Venta_hoy,Dev_proxima,Demanda_uni_equil,lag0tar1,A,B,C,lag1tar1,SA21,L21
24,9,1110,7,3302,15766,35651,12,90.0,0.0,2.564949,5.0,5,1.766936,2.0,1.0,1.0,2.564949
25,9,1110,7,3303,15766,35651,12,90.0,0.0,2.564949,3.0,3,2.190761,1.0,NaN,NaN,NaN
26,9,1110,7,3303,15766,35651,12,90.0,0.0,2.564949,3.0,3,2.190761,1.0,NaN,NaN,NaN
27,9,1110,7,3305,15766,35651,12,90.0,0.0,2.564949,1.0,1,2.564949,1.0,1.0,1.0,2.564949


Count of missing numbers after L21 in validation part 1 in column lag1tar1 is 1
Count of missing numbers after L21 in validation part 2 in column lag1tar1 is 2


,Semana,Agencia_ID,Canal_ID,Ruta_SAK,Cliente_ID,Producto_ID,Venta_uni_hoy,Venta_hoy,Dev_proxima,Demanda_uni_equil,lag0tar1,A,B,C,lag1tar1,SA21,L21
0,3,1110,7,3301,15766,1212,3,25.139999,0.0,1.386294,4.0,4,1.744863,1.75,NaN,NaN,NaN
1,3,1110,7,3302,15766,1216,4,33.520000,0.0,1.609438,5.0,5,1.766936,2.00,NaN,NaN,NaN
2,3,1110,7,3303,15766,1238,4,39.320000,0.0,1.609438,3.0,3,2.190761,1.00,NaN,NaN,NaN
3,3,1110,7,3301,15766,1240,4,33.520000,0.0,1.609438,4.0,4,1.744863,1.75,NaN,NaN,NaN
4,3,1110,7,3302,15766,30548,3,22.920000,0.0,1.386294,5.0,5,1.766936,2.00,NaN,NaN,NaN
5,4,1110,7,3302,15766,1250,5,38.200001,0.0,1.791759,5.0,5,1.766936,2.00,NaN,NaN,NaN
6,4,1110,7,3302,15766,1216,3,20.280001,0.0,1.386294,5.0,5,1.766936,2.00,NaN,NaN,NaN
7,4,1110,7,3301,15766,1216,6,56.099998,0.0,1.945910,4.0,4,1.744863,1.75,NaN,NaN,NaN
8,4,1110,7,3301,15766,4085,4,24.600000,0.0,1.609438,4.0,4,1.744863,1.75,NaN,NaN,NaN
9,5,1110,7,3302,15766,1216,6,31.680000,0.0,1.945910,5.0,5,1.766936,2.00,2.0,2.0,1.497866


,Semana,Agencia_ID,Canal_ID,Ruta_SAK,Cliente_ID,Producto_ID,Venta_uni_hoy,Venta_hoy,Dev_proxima,Demanda_uni_equil,lag0tar1,A,B,C,lag1tar1,SA21,L21
21,8,1110,7,3305,15766,35651,12,90.0,0.0,2.564949,1.0,1,2.564949,1.00,NaN,NaN,NaN
22,8,1110,7,3301,15766,35651,12,90.0,0.0,2.564949,4.0,4,1.744863,1.75,1.0,1.0,2.079442
23,8,1110,7,3302,15766,35651,12,90.0,0.0,2.564949,5.0,5,1.766936,2.00,2.0,2.0,1.589027


,Semana,Agencia_ID,Canal_ID,Ruta_SAK,Cliente_ID,Producto_ID,Venta_uni_hoy,Venta_hoy,Dev_proxima,Demanda_uni_equil,lag0tar1,A,B,C,lag1tar1,SA21,L21
24,9,1110,7,3302,15766,35651,12,90.0,0.0,2.564949,5.0,5,1.766936,2.0,1.0,1.0,2.564949
25,9,1110,7,3303,15766,35651,12,90.0,0.0,2.564949,3.0,3,2.190761,1.0,NaN,NaN,NaN
26,9,1110,7,3303,15766,35651,12,90.0,0.0,2.564949,3.0,3,2.190761,1.0,NaN,NaN,NaN
27,9,1110,7,3305,15766,35651,12,90.0,0.0,2.564949,1.0,1,2.564949,1.0,1.0,1.0,2.564949


## Test 3

In [95]:
FE.AddConfigurableFeaturesToTrain(configLag1Target1)

SA2 is not in columns..


SA2
Semana Ruta_SAK     
4      3301        2
       3302        2
       3303        1
5      3301        2
       3302        2
6      3302        3
       3303        1
7      3301        1
       3302        2
       3303        1
8      3301        2
       3302        1
       3305        1

,Semana,Agencia_ID,Canal_ID,Ruta_SAK,Cliente_ID,Producto_ID,Venta_uni_hoy,Venta_hoy,Dev_proxima,Demanda_uni_equil,lag0tar1,A,B,C,lag1tar1,SA21,L21,SA2
0,3,1110,7,3301,15766,1212,3,25.139999,0.0,1.386294,4.0,4,1.744863,1.75,NaN,NaN,NaN,NaN
1,3,1110,7,3302,15766,1216,4,33.520000,0.0,1.609438,5.0,5,1.766936,2.00,NaN,NaN,NaN,NaN
2,3,1110,7,3303,15766,1238,4,39.320000,0.0,1.609438,3.0,3,2.190761,1.00,NaN,NaN,NaN,NaN
3,3,1110,7,3301,15766,1240,4,33.520000,0.0,1.609438,4.0,4,1.744863,1.75,NaN,NaN,NaN,NaN
4,3,1110,7,3302,15766,30548,3,22.920000,0.0,1.386294,5.0,5,1.766936,2.00,NaN,NaN,NaN,NaN
5,4,1110,7,3302,15766,1250,5,38.200001,0.0,1.791759,5.0,5,1.766936,2.00,NaN,NaN,NaN,2.0
6,4,1110,7,3302,15766,1216,3,20.280001,0.0,1.386294,5.0,5,1.766936,2.00,NaN,NaN,NaN,2.0
7,4,1110,7,3301,15766,1216,6,56.099998,0.0,1.945910,4.0,4,1.744863,1.75,NaN,NaN,NaN,2.0
8,4,1110,7,3301,15766,4085,4,24.600000,0.0,1.609438,4.0,4,1.744863,1.75,NaN,NaN,NaN,2.0
9,5,1110,7,3302,15766,1216,6,31.680000,0.0,1.945910,5.0,5,1.766936,2.00,2.0,2.0,1.497866,2.0


,Semana,Agencia_ID,Canal_ID,Ruta_SAK,Cliente_ID,Producto_ID,Venta_uni_hoy,Venta_hoy,Dev_proxima,Demanda_uni_equil,lag0tar1,A,B,C,lag1tar1,SA21,L21,SA2
21,8,1110,7,3305,15766,35651,12,90.0,0.0,2.564949,1.0,1,2.564949,1.00,NaN,NaN,NaN,1
22,8,1110,7,3301,15766,35651,12,90.0,0.0,2.564949,4.0,4,1.744863,1.75,1.0,1.0,2.079442,2
23,8,1110,7,3302,15766,35651,12,90.0,0.0,2.564949,5.0,5,1.766936,2.00,2.0,2.0,1.589027,1


,Semana,Agencia_ID,Canal_ID,Ruta_SAK,Cliente_ID,Producto_ID,Venta_uni_hoy,Venta_hoy,Dev_proxima,Demanda_uni_equil,lag0tar1,A,B,C,lag1tar1,SA21,L21
24,9,1110,7,3302,15766,35651,12,90.0,0.0,2.564949,5.0,5,1.766936,2.0,1.0,1.0,2.564949
25,9,1110,7,3303,15766,35651,12,90.0,0.0,2.564949,3.0,3,2.190761,1.0,NaN,NaN,NaN
26,9,1110,7,3303,15766,35651,12,90.0,0.0,2.564949,3.0,3,2.190761,1.0,NaN,NaN,NaN
27,9,1110,7,3305,15766,35651,12,90.0,0.0,2.564949,1.0,1,2.564949,1.0,1.0,1.0,2.564949


...

,Semana,Agencia_ID,Canal_ID,Ruta_SAK,Cliente_ID,Producto_ID,Venta_uni_hoy,Venta_hoy,Dev_proxima,Demanda_uni_equil,lag0tar1,A,B,C,lag1tar1,SA21,L21
24,9,1110,7,3302,15766,35651,12,90.0,0.0,2.564949,5.0,5,1.766936,2.0,1.0,1.0,2.564949
25,9,1110,7,3303,15766,35651,12,90.0,0.0,2.564949,3.0,3,2.190761,1.0,NaN,NaN,NaN
26,9,1110,7,3303,15766,35651,12,90.0,0.0,2.564949,3.0,3,2.190761,1.0,NaN,NaN,NaN
27,9,1110,7,3305,15766,35651,12,90.0,0.0,2.564949,1.0,1,2.564949,1.0,1.0,1.0,2.564949


Count of missing numbers after L in validation part 1 in column lag1tar1 is 0
Count of missing numbers after L in validation part 2 in column lag1tar1 is 2


,Semana,Agencia_ID,Canal_ID,Ruta_SAK,Cliente_ID,Producto_ID,Venta_uni_hoy,Venta_hoy,Dev_proxima,Demanda_uni_equil,lag0tar1,A,B,C,lag1tar1,SA21,L21,SA2,L
0,3,1110,7,3301,15766,1212,3,25.139999,0.0,1.386294,4.0,4,1.744863,1.75,NaN,NaN,NaN,NaN,NaN
1,3,1110,7,3302,15766,1216,4,33.520000,0.0,1.609438,5.0,5,1.766936,2.00,NaN,NaN,NaN,NaN,NaN
2,3,1110,7,3303,15766,1238,4,39.320000,0.0,1.609438,3.0,3,2.190761,1.00,NaN,NaN,NaN,NaN,NaN
3,3,1110,7,3301,15766,1240,4,33.520000,0.0,1.609438,4.0,4,1.744863,1.75,NaN,NaN,NaN,NaN,NaN
4,3,1110,7,3302,15766,30548,3,22.920000,0.0,1.386294,5.0,5,1.766936,2.00,NaN,NaN,NaN,NaN,NaN
5,4,1110,7,3302,15766,1250,5,38.200001,0.0,1.791759,5.0,5,1.766936,2.00,2.0,NaN,NaN,2.0,1.497866
6,4,1110,7,3302,15766,1216,3,20.280001,0.0,1.386294,5.0,5,1.766936,2.00,2.0,NaN,NaN,2.0,1.497866
7,4,1110,7,3301,15766,1216,6,56.099998,0.0,1.945910,4.0,4,1.744863,1.75,2.0,NaN,NaN,2.0,1.497866
8,4,1110,7,3301,15766,4085,4,24.600000,0.0,1.609438,4.0,4,1.744863,1.75,2.0,NaN,NaN,2.0,1.497866
9,5,1110,7,3302,15766,1216,6,31.680000,0.0,1.945910,5.0,5,1.766936,2.00,2.0,2.0,1.497866,2.0,1.589027


,Semana,Agencia_ID,Canal_ID,Ruta_SAK,Cliente_ID,Producto_ID,Venta_uni_hoy,Venta_hoy,Dev_proxima,Demanda_uni_equil,lag0tar1,A,B,C,lag1tar1,SA21,L21,SA2,L
21,8,1110,7,3305,15766,35651,12,90.0,0.0,2.564949,1.0,1,2.564949,1.00,1.0,NaN,NaN,1,2.564949
22,8,1110,7,3301,15766,35651,12,90.0,0.0,2.564949,4.0,4,1.744863,1.75,1.0,1.0,2.079442,2,1.791759
23,8,1110,7,3302,15766,35651,12,90.0,0.0,2.564949,5.0,5,1.766936,2.00,2.0,2.0,1.589027,1,2.564949


,Semana,Agencia_ID,Canal_ID,Ruta_SAK,Cliente_ID,Producto_ID,Venta_uni_hoy,Venta_hoy,Dev_proxima,Demanda_uni_equil,lag0tar1,A,B,C,lag1tar1,SA21,L21
24,9,1110,7,3302,15766,35651,12,90.0,0.0,2.564949,5.0,5,1.766936,2.0,1.0,1.0,2.564949
25,9,1110,7,3303,15766,35651,12,90.0,0.0,2.564949,3.0,3,2.190761,1.0,NaN,NaN,NaN
26,9,1110,7,3303,15766,35651,12,90.0,0.0,2.564949,3.0,3,2.190761,1.0,NaN,NaN,NaN
27,9,1110,7,3305,15766,35651,12,90.0,0.0,2.564949,1.0,1,2.564949,1.0,1.0,1.0,2.564949


## Test 4

In [120]:
FE.AddConfigurableFeaturesToTrain(configLag1Target1F)

SAP1CC is not in columns..


SAP1CC
Semana Ruta_SAK Producto_ID        
4      3301     1212              1
                1240              1
       3302     1216              1
                30548             1
       3303     1238              1
5      3301     1216              1
                4085              1
       3302     1216              1
                1250              1
6      3302     1216              2
                30548             1
       3303     30548             1
7      3301     31309             1
       3302     32393             1
                32933             1
       3303     1212              1
8      3301     1212              1
                34053             1
       3302     35651             1
       3305     35651             1

,Semana,Agencia_ID,Canal_ID,Ruta_SAK,Cliente_ID,Producto_ID,Venta_uni_hoy,Venta_hoy,Dev_proxima,Demanda_uni_equil,Lag1Target1F,SAP1CC
0,3,1110,7,3301,15766,1212,3,25.139999,0.0,1.386294,NaN,NaN
1,3,1110,7,3302,15766,1216,4,33.520000,0.0,1.609438,NaN,NaN
2,3,1110,7,3303,15766,1238,4,39.320000,0.0,1.609438,NaN,NaN
3,3,1110,7,3301,15766,1240,4,33.520000,0.0,1.609438,NaN,NaN
4,3,1110,7,3302,15766,30548,3,22.920000,0.0,1.386294,NaN,NaN
5,4,1110,7,3302,15766,1250,5,38.200001,0.0,1.791759,NaN,NaN
6,4,1110,7,3302,15766,1216,3,20.280001,0.0,1.386294,NaN,1.0
7,4,1110,7,3301,15766,1216,6,56.099998,0.0,1.945910,NaN,NaN
8,4,1110,7,3301,15766,4085,4,24.600000,0.0,1.609438,NaN,NaN
9,5,1110,7,3302,15766,1216,6,31.680000,0.0,1.945910,NaN,1.0


,Semana,Agencia_ID,Canal_ID,Ruta_SAK,Cliente_ID,Producto_ID,Venta_uni_hoy,Venta_hoy,Dev_proxima,Demanda_uni_equil,Lag1Target1F,SAP1CC
21,8,1110,7,3305,15766,35651,12,90.0,0.0,2.564949,NaN,1.0
22,8,1110,7,3301,15766,35651,12,90.0,0.0,2.564949,NaN,NaN
23,8,1110,7,3302,15766,35651,12,90.0,0.0,2.564949,NaN,1.0


,Semana,Agencia_ID,Canal_ID,Ruta_SAK,Cliente_ID,Producto_ID,Venta_uni_hoy,Venta_hoy,Dev_proxima,Demanda_uni_equil
24,9,1110,7,3302,15766,35651,12,90.0,0.0,2.564949
25,9,1110,7,3303,15766,35651,12,90.0,0.0,2.564949
26,9,1110,7,3303,15766,35651,12,90.0,0.0,2.564949
27,9,1110,7,3305,15766,35651,12,90.0,0.0,2.564949


...

,Semana,Agencia_ID,Canal_ID,Ruta_SAK,Cliente_ID,Producto_ID,Venta_uni_hoy,Venta_hoy,Dev_proxima,Demanda_uni_equil
24,9,1110,7,3302,15766,35651,12,90.0,0.0,2.564949
25,9,1110,7,3303,15766,35651,12,90.0,0.0,2.564949
26,9,1110,7,3303,15766,35651,12,90.0,0.0,2.564949
27,9,1110,7,3305,15766,35651,12,90.0,0.0,2.564949


Count of missing numbers after SAP1MM in validation part 1 in column Lag1Target1F is 1


,Semana,Agencia_ID,Canal_ID,Ruta_SAK,Cliente_ID,Producto_ID,Venta_uni_hoy,Venta_hoy,Dev_proxima,Demanda_uni_equil,Lag1Target1F
0,3,1110,7,3301,15766,1212,3,25.139999,0.0,1.386294,NaN
1,3,1110,7,3302,15766,1216,4,33.520000,0.0,1.609438,NaN
2,3,1110,7,3303,15766,1238,4,39.320000,0.0,1.609438,NaN
3,3,1110,7,3301,15766,1240,4,33.520000,0.0,1.609438,NaN
4,3,1110,7,3302,15766,30548,3,22.920000,0.0,1.386294,NaN
5,4,1110,7,3302,15766,1250,5,38.200001,0.0,1.791759,NaN
6,4,1110,7,3302,15766,1216,3,20.280001,0.0,1.386294,1.0
7,4,1110,7,3301,15766,1216,6,56.099998,0.0,1.945910,NaN
8,4,1110,7,3301,15766,4085,4,24.600000,0.0,1.609438,NaN
9,5,1110,7,3302,15766,1216,6,31.680000,0.0,1.945910,1.0


,Semana,Agencia_ID,Canal_ID,Ruta_SAK,Cliente_ID,Producto_ID,Venta_uni_hoy,Venta_hoy,Dev_proxima,Demanda_uni_equil,Lag1Target1F
21,8,1110,7,3305,15766,35651,12,90.0,0.0,2.564949,1.0
22,8,1110,7,3301,15766,35651,12,90.0,0.0,2.564949,NaN
23,8,1110,7,3302,15766,35651,12,90.0,0.0,2.564949,1.0


,Semana,Agencia_ID,Canal_ID,Ruta_SAK,Cliente_ID,Producto_ID,Venta_uni_hoy,Venta_hoy,Dev_proxima,Demanda_uni_equil
24,9,1110,7,3302,15766,35651,12,90.0,0.0,2.564949
25,9,1110,7,3303,15766,35651,12,90.0,0.0,2.564949
26,9,1110,7,3303,15766,35651,12,90.0,0.0,2.564949
27,9,1110,7,3305,15766,35651,12,90.0,0.0,2.564949


## Test setup..

In [22]:
parameterDict = {"ValidationStart":6, 
    "ValidationEnd":7,
    "maxLag":2,    
    "trainHdfPath":'../../input/train_100.h5',
    "trainHdfFile":"train",
    "testHdfPath":"../../input/test.h5",
    "testHdfFile":"test", 
    "trainTypes" : {'Semana':np.uint8, 'Agencia_ID':np.uint16, 'Canal_ID':np.uint8,'Ruta_SAK':np.uint16, 
        'Cliente_ID':np.uint32, 'Producto_ID':np.uint16,'Venta_uni_hoy':np.uint16, 'Venta_hoy':np.float32,
                    'Dev_proxima':np.float32,'Demanda_uni_equil':np.uint32}, 
    "testTypes" : {'id':np.uint32,'Semana':np.uint8, 'Agencia_ID':np.uint16, 'Canal_ID':np.uint8,'Ruta_SAK':np.uint16,
        'Cliente_ID':np.uint32, 'Producto_ID':np.uint16},
    "trainCsvPath":'../../input/train_100.csv'   ,
    "testCsvPath":'../../input/test.csv'}

FE = FeatureEngineering(**parameterDict)
print FE.__dict__

{'maxLag': 2, 'testTypes': {'Cliente_ID': <type 'numpy.uint32'>, 'Ruta_SAK': <type 'numpy.uint16'>, 'Canal_ID': <type 'numpy.uint8'>, 'Producto_ID': <type 'numpy.uint16'>, 'Agencia_ID': <type 'numpy.uint16'>, 'Semana': <type 'numpy.uint8'>, 'id': <type 'numpy.uint32'>}, 'testHdfFile': 'test', 'trainTypes': {'Cliente_ID': <type 'numpy.uint32'>, 'Demanda_uni_equil': <type 'numpy.uint32'>, 'Producto_ID': <type 'numpy.uint16'>, 'Agencia_ID': <type 'numpy.uint16'>, 'Dev_proxima': <type 'numpy.float32'>, 'Semana': <type 'numpy.uint8'>, 'Ruta_SAK': <type 'numpy.uint16'>, 'Canal_ID': <type 'numpy.uint8'>, 'Venta_uni_hoy': <type 'numpy.uint16'>, 'Venta_hoy': <type 'numpy.float32'>}, 'trainCsvPath': '../../input/train_100.csv', 'testHdfPath': '../../input/test.h5', 'testCsvPath': '../../input/test.csv', 'ValidationStart': 6, 'trainHdfFile': 'train', 'trainHdfPath': '../../input/train_100.h5', 'ValidationSmallerThan': 7}


In [23]:
#FE.ReadCsv('train')
#FE.Preprocess('train', [["Demanda_uni_equil",np.log1p,'float32']])
#FE.SaveDataFrameToHdf('train')

,Semana,Agencia_ID,Canal_ID,Ruta_SAK,Cliente_ID,Producto_ID,Venta_uni_hoy,Venta_hoy,Dev_proxima,Demanda_uni_equil
0,3,1111,1,4402,50789,32846,2,10.920000,0,2
1,3,1112,1,2103,1209569,36748,4,46.119999,0,4


<class 'pandas.core.frame.DataFrame'>
Int64Index: 18 entries, 0 to 17
Data columns (total 10 columns):
Semana               18 non-null uint8
Agencia_ID           18 non-null uint16
Canal_ID             18 non-null uint8
Ruta_SAK             18 non-null uint16
Cliente_ID           18 non-null uint32
Producto_ID          18 non-null uint16
Venta_uni_hoy        18 non-null uint16
Venta_hoy            18 non-null float32
Dev_proxima          18 non-null float32
Demanda_uni_equil    18 non-null uint32
dtypes: float32(2), uint16(4), uint32(2), uint8(2)
memory usage: 612.0 bytes
None


In [24]:
FE.ReadHdf('train')

,Semana,Agencia_ID,Canal_ID,Ruta_SAK,Cliente_ID,Producto_ID,Venta_uni_hoy,Venta_hoy,Dev_proxima,Demanda_uni_equil
0,3,1111,1,4402,50789,32846,2,10.920000,0,1.098612
1,3,1112,1,2103,1209569,36748,4,46.119999,0,1.609438


<class 'pandas.core.frame.DataFrame'>
Int64Index: 18 entries, 0 to 17
Data columns (total 10 columns):
Semana               18 non-null uint8
Agencia_ID           18 non-null uint16
Canal_ID             18 non-null uint8
Ruta_SAK             18 non-null uint16
Cliente_ID           18 non-null uint32
Producto_ID          18 non-null uint16
Venta_uni_hoy        18 non-null uint16
Venta_hoy            18 non-null float32
Dev_proxima          18 non-null float32
Demanda_uni_equil    18 non-null float32
dtypes: float32(3), uint16(4), uint32(1), uint8(2)
memory usage: 540.0 bytes
None


In [12]:
display(FE.AddDemandaGeneralMean())

,Semana,Agencia_ID,Canal_ID,Ruta_SAK,Cliente_ID,Producto_ID,Demanda_uni_equil,DemandaGeneralMean
0,3,1,7,3301,15766,1200,1.386294,5.159912
1,3,2,7,3301,15766,1200,1.609438,5.159912
2,3,1,7,3301,15766,1200,3.713572,5.159912
3,3,2,7,3301,15766,1300,1.098612,5.159912
4,3,3,7,3301,15900,1300,1.386294,5.159912
5,3,2,7,3301,15766,1300,6.908755,5.159912
6,4,3,7,3301,15766,1200,11.512936,5.159912
7,4,2,7,3301,15766,1300,11.512936,5.159912
8,4,1,7,3301,15766,1300,11.512936,5.159912
9,4,4,7,3301,15900,1400,11.512936,5.159912


None

In [28]:
qw = FE.train.loc[FE.train.loc[:,'Semana']>4]
qw

,Semana,Agencia_ID,Canal_ID,Ruta_SAK,Cliente_ID,Producto_ID,Demanda_uni_equil,DemandaGeneralMean,lag2A,lag2Ca,lag2R,lag2Cl,lag2P,lag2S
10,5,1110,7,3301,15766,1200,2.397895,5.159912,2.683828,2.683828,2.683828,2.943334,2.236435,2.683828
11,5,1110,7,3301,15766,1200,2.397895,5.159912,2.683828,2.683828,2.683828,2.943334,2.236435,2.683828
12,5,1110,7,3301,15766,1200,2.890372,5.159912,2.683828,2.683828,2.683828,2.943334,2.236435,2.683828
13,5,1110,7,3301,15900,1400,2.397895,5.159912,2.683828,2.683828,2.683828,1.386294,NaN,2.683828
14,6,1110,7,3301,15766,1400,5.298317,5.159912,11.512936,11.512936,11.512936,11.512936,11.512936,11.512936


In [79]:
FE.train.loc[:,'target'] = np.nan
FE.train

,Semana,Agencia_ID,Canal_ID,Ruta_SAK,Cliente_ID,Producto_ID,Demanda_uni_equil,DemandaGeneralMean,lag2A,lag2Ca,lag2R,lag2Cl,lag2P,lag2S,target
0,3,1110,7,3301,15766,1200,1.386294,5.159912,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,3,1110,7,3301,15766,1200,1.609438,5.159912,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,3,1110,7,3301,15766,1200,3.713572,5.159912,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3,1110,7,3301,15766,1300,1.098612,5.159912,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,3,1110,7,3301,15900,1300,1.386294,5.159912,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,3,1110,7,3301,15766,1300,6.908755,5.159912,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,4,1110,7,3301,15766,1200,11.512936,5.159912,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,4,1110,7,3301,15766,1300,11.512936,5.159912,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,4,1110,7,3301,15766,1300,11.512936,5.159912,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,4,1110,7,3301,15900,1400,11.512936,5.159912,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [82]:
FE.train.loc[pd.isnull(FE.train['target']), 'target'] = FE.train.loc[pd.isnull(FE.train['target']), 'lag2P']
FE.train["target"]

0           NaN
1           NaN
2           NaN
3           NaN
4           NaN
5           NaN
6           NaN
7           NaN
8           NaN
9           NaN
10     2.236435
11     2.236435
12     2.236435
13          NaN
14    11.512936
Name: target, dtype: float32

In [87]:
FE.train.loc[FE.train['Semana']>4,"target"].isnull().sum()

10     2.236435
11     2.236435
12     2.236435
13          NaN
14    11.512936
Name: target, dtype: float32

In [88]:
FE.train.loc[FE.train['Semana']>4,"target"].isnull().sum()

1

In [89]:
FE.train.loc[pd.isnull(FE.train['target']), 'target'] = FE.train.loc[pd.isnull(FE.train['target']), 'lag2Cl']
FE.train["target"]

0           NaN
1           NaN
2           NaN
3           NaN
4           NaN
5           NaN
6           NaN
7           NaN
8           NaN
9           NaN
10     2.236435
11     2.236435
12     2.236435
13     1.386294
14    11.512936
Name: target, dtype: float32

In [90]:
FE.train.loc[FE.train['Semana']>4,"target"].isnull().sum()

0

In [45]:
"Semana" not in FE.train.columns

False

In [46]:
FE.train

,Semana,Agencia_ID,Canal_ID,Ruta_SAK,Cliente_ID,Producto_ID,Demanda_uni_equil,lag0par1,lag2par1
0,3,1,7,3301,15766,1200,1.386294,5.537601,NaN
1,3,2,7,3301,15766,1200,1.609438,4.320922,NaN
2,3,1,7,3301,15766,1200,3.713572,5.537601,NaN
3,3,2,7,3301,15766,1300,1.098612,4.320922,NaN
4,3,3,7,3301,15900,1300,1.386294,4.546874,NaN
5,3,2,7,3301,15766,1300,6.908755,4.320922,NaN
6,4,3,7,3301,15766,1200,11.512936,4.546874,NaN
7,4,2,7,3301,15766,1300,11.512936,4.320922,NaN
8,4,1,7,3301,15766,1300,11.512936,5.537601,NaN
9,4,4,7,3301,15900,1400,11.512936,11.512936,NaN


In [64]:
print "{} haha {}".format(1,2)

1 haha 2


In [25]:
for i in FE.train.groupby('Canal_ID')['Demanda_uni_equil']:
    print i

(2, 13    2.397895
14    5.298317
Name: Demanda_uni_equil, dtype: float32)
(7, 0      1.386294
1      1.609438
2      3.713572
3      1.098612
4      1.386294
5      6.908755
6     11.512936
7     11.512936
8     11.512936
9     11.512936
10     2.397895
11     2.397895
12     2.890372
15     2.995732
17     4.595120
18     5.298317
Name: Demanda_uni_equil, dtype: float32)
(10, 16    5.288267
Name: Demanda_uni_equil, dtype: float32)


In [29]:
FE.train[['Canal_ID','Demanda_uni_equil']]

,Canal_ID,Demanda_uni_equil
0,7,1.386294
1,7,1.609438
2,7,3.713572
3,7,1.098612
4,7,1.386294
5,7,6.908755
6,7,11.512936
7,7,11.512936
8,7,11.512936
9,7,11.512936


In [31]:
["Cliente_ID","Producto_ID"] + 'abc'

TypeError: can only concatenate list (not "str") to list

In [58]:
import xgboost as xgb

params = {}
params['objective'] = "reg:linear"
params['eta'] = 0.1
params['max_depth'] = 5
params['subsample'] = 0.8
params['colsample_bytree'] = 0.6
params['silent'] = True
'alpha': 0.0001, 'lambda': 1

SyntaxError: invalid syntax (<ipython-input-58-818fa5b8aef9>, line 10)

In [59]:
import xgboost as xgb

ImportError: No module named xgboost

In [28]:
regres = xgb.XGBRegressor()

In [29]:
regres

XGBRegressor(base_score=0.5, colsample_bylevel=1, colsample_bytree=1, gamma=0,
       learning_rate=0.1, max_delta_step=0, max_depth=3,
       min_child_weight=1, missing=None, n_estimators=100, nthread=-1,
       objective='reg:linear', reg_alpha=0, reg_lambda=1,
       scale_pos_weight=1, seed=0, silent=True, subsample=1)